# Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week 1)  
## Shu Yoko
### Aug 9

Now that you have been equipped with the skills and the tools to use location data to explore a geographical location, over the course of two weeks, you will have the opportunity to be as creative as you want and come up with an idea to leverage the Foursquare location data to explore or compare neighborhoods or cities of your choice or to come up with a problem that you can use the Foursquare location data to solve. If you cannot think of an idea or a problem, here are some ideas to get you started:

1. In Module 3, we explored New York City and the city of Toronto and segmented and clustered their neighborhoods. Both cities are very diverse and are the financial capitals of their respective countries. One interesting idea would be to compare the neighborhoods of the two cities and determine how similar or dissimilar they are. Is New York City more like Toronto or Paris or some other multicultural city? I will leave it to you to refine this idea.
2. In a city of your choice, if someone is looking to open a restaurant, where would you recommend that they open it? Similarly, if a contractor is trying to start their own business, where would you recommend that they setup their office?

These are just a couple of many ideas and problems that can be solved using location data in addition to other datasets. No matter what you decide to do, make sure to provide sufficient justification of why you think what you want to do or solve is important and why would a client or a group of people be interested in your project.

In [3]:
#!pip install bs4
#!pip install geopy
#!pip install folium==0.5.0
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library
# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Week 1 

For this week, you will required to submit the following:

A description of the problem and a discussion of the background. (15 marks)
A description of the data and how it will be used to solve the problem. (15 marks)

## 1. Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.

In my project, I will run the hypothesis test about the **hotelling's law** to see if it's actually decent economical strategies. 

Hotelling Law is that: 
*Suppose there are two competing shops located along the length of a street running north and south, with customers spread equally along the street. Both shop owners want their shops to be where they will get most market share of customers. If both shops sell the same range of goods at the same prices then the locations of the shops are themselves the 'products'. Each customer will always choose the nearer shop as it is disadvantageous to travel to the farther. - Wikipedia*. 
[For more discription](https://en.wikipedia.org/wiki/Hotelling's_law)

Basically, this theory argues that if the shops offer the similar products, the shop with most dominant area will win. Here is some figure that explain. Let's say there are two stores A and B. If they decided to open their shop in a linear city that has 12 dashes of land.  

|------|------|   

First, they build the shop as they can share same amount of customer. They both win ------ (6 dashes) of the area/customers that are coming from.    
|---A---|---B---|   

But if B moves 2 dashes closer to shop A.  

|---A--**-|-B-----**|   

Shop B wins some of A's customers since they sell similar products and customer choose to go to the closer shop. A loses some of its customers.  

So A decided to move towards shop B, so A can win some customers too.  

|------A|B------|  

Here, both shops end up with locating at the center.  

Likewise, this theory argues that if the shop was located away from the center, they will loose the competition.  
|--A--**--|-B-----**|  

In this project, I will test if this theory is correct at the city in Manhattan.  
The reason why I chose Manhattan for this project is that Manhattan is a island like shape and is isolated from other land. So I thought that it is easy to calculate the center of the Manhattan.  

## 2. Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.

I will use Forsquare data in Manhattan. For all pizza restaurants (stc) in Manhattan with similar rate and similar price, I will see if the pizza restaurants in the near the center of Manhattan has more reviews (# of customers).     

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


#pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


#! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
CLIENT_ID = 'USWT23SP2JAKIN4FPXBP1JNHW3BNMMGD42YV2FKSQFYUVQIM' # your Foursquare ID
CLIENT_SECRET = 'LUB4B1HJET4EAXBHJG5APWOHBYQ13HXTBTWABOP1TZL2EEJT' # your Foursquare Secret
ACCESS_TOKEN = 'RM41FCWJOEZTN0LJOJJJMI3FHVOUQSOLHAHHORHBSG205TXY' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: USWT23SP2JAKIN4FPXBP1JNHW3BNMMGD42YV2FKSQFYUVQIM
CLIENT_SECRET:LUB4B1HJET4EAXBHJG5APWOHBYQ13HXTBTWABOP1TZL2EEJT


In [3]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


In [4]:
search_query = 'Pizza'
radius = 500
print(search_query + ' .... OK!')

Pizza .... OK!


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=USWT23SP2JAKIN4FPXBP1JNHW3BNMMGD42YV2FKSQFYUVQIM&client_secret=LUB4B1HJET4EAXBHJG5APWOHBYQ13HXTBTWABOP1TZL2EEJT&ll=40.7149555,-74.0153365&oauth_token=RM41FCWJOEZTN0LJOJJJMI3FHVOUQSOLHAHHORHBSG205TXY&v=20180604&query=Pizza&radius=500&limit=30'

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6113c570ba16570876af6f7e'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '57acc558498e036ce8f2b0b4',
    'name': 'La Pizza & La Pasta',
    'location': {'address': '4 World Trade Ctr',
     'crossStreet': 'Liberty St',
     'lat': 40.710566583744715,
     'lng': -74.01249396848024,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.710566583744715,
       'lng': -74.01249396848024}],
     'distance': 544,
     'postalCode': '10007',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['4 World Trade Ctr (Liberty St)',
      'New York, NY 10007']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primar

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,57acc558498e036ce8f2b0b4,La Pizza & La Pasta,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1628685680,False,4 World Trade Ctr,Liberty St,40.710567,-74.012494,"[{'label': 'display', 'lat': 40.71056658374471...",...,New York,NY,United States,"[4 World Trade Ctr (Liberty St), New York, NY ...",NaN,NaN,NaN,NaN,NaN,NaN
1,4bc238adf8219c744286b410,Amore's Pizza Restaurant,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1628685680,False,147 Chambers St,Hudson Street,40.715860,-74.009888,"[{'label': 'display', 'lat': 40.71585960614924...",...,New York,NY,United States,"[147 Chambers St (Hudson Street), New York, NY...",1431324,https://www.seamless.com/menu/cafe-amores-pizz...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png
2,4c35ffa0a0ced13aa6aa1a6e,Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1628685680,False,154 Chambers St,NaN,40.715859,-74.009889,"[{'label': 'display', 'lat': 40.71585918421961...",...,New York,NY,United States,"[154 Chambers St, New York, NY 10007]",NaN,NaN,NaN,NaN,NaN,NaN
3,4fa862b3e4b0ebff2f749f06,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1628685680,False,225 Murray St,NaN,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",...,New York,NY,United States,"[225 Murray St, New York, NY 10282]",NaN,NaN,NaN,NaN,NaN,NaN
4,4bf6e92e5efe2d7fb1016834,Pronto Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1628685680,False,114 Liberty St,church,40.710018,-74.012418,"[{'label': 'display', 'lat': 40.71001795396908...",...,New York,NY,United States,"[114 Liberty St (church), New York, NY 10006]",304951,https://www.seamless.com/menu/pronto-pizza-114...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png
